In [14]:
import alborex_functions
import alborexdata
import netCDF4
import glob
import os
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap 
import numpy as np
import datetime
from importlib import reload
%matplotlib inline

In [15]:
reload(alborex_functions)

<module 'alborex_functions' from '/home/ctroupin/Publis/201703_AlborexData/python/alborex_functions.py'>

In [16]:
import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)

# Configuration

Creation of figures

In [17]:
makeFigs, makeFigsLeaflet = 1, 1

Create a logger

In [18]:
logger = alborexdata.configure_logging()

Domain

In [19]:
#coordinates = [-6.75, 3.001, 34.75, 40.]  # AlborEx
coordinates = [-5.5, 1.3, 34.9, 38.9]      # ALMO

Create a list of netCDF files containing SST

In [20]:
"""
# AlborEx
figdir = "../figures/SST/"
figdir2 = "/home/ctroupin/Projects/Leaflet/AlborEx/images/"
sstdir = "/home/ctroupin/DataOceano/Alborex/SST/"
"""
# ALMO
figdir = "/home/ctroupin/Publis/201702_ALMO/figures/"
sstdir = "/home/ctroupin/DataOceano/ALMO/SST/"
figdir2 = "/home/ctroupin/Projects/Leaflet/Missions/ALMO/"

In [21]:
coastfile = "../data/coastline_cartex_f3.txt"
sstfilelist = sorted(glob.glob(os.path.join(sstdir, '*SST*.nc')))
nfiles = len(sstfilelist)
logger.info("Working on {0} SST files".format(nfiles))

2017-03-13 14:08:07,113 - alborex_logger - INFO - Working on 22 SST files
2017-03-13 14:08:07,113 - alborex_logger - INFO - Working on 22 SST files
2017-03-13 14:08:07,113 - alborex_logger - INFO - Working on 22 SST files


In [22]:
cmap = plt.cm.RdYlBu_r

# Data reading and plotting

Make figure directory if necessary.

In [23]:
if os.path.isdir(figdir):
    logger.debug("Figure directory {0} already exists".format(figdir))
else:
    os.makedirs(figdir)
    logger.debug("Creating figure directory {0}".format(figdir))
    
if os.path.isdir(figdir2):
    logger.debug("Figure directory {0} already exists".format(figdir))
else:
    os.makedirs(figdir2)
    logger.debug("Creating figure directory {0}".format(figdir))

2017-03-13 14:08:09,186 - alborex_logger - DEBUG - Figure directory /home/ctroupin/Publis/201702_ALMO/figures/ already exists
2017-03-13 14:08:09,186 - alborex_logger - DEBUG - Figure directory /home/ctroupin/Publis/201702_ALMO/figures/ already exists
2017-03-13 14:08:09,186 - alborex_logger - DEBUG - Figure directory /home/ctroupin/Publis/201702_ALMO/figures/ already exists
2017-03-13 14:08:09,191 - alborex_logger - DEBUG - Figure directory /home/ctroupin/Publis/201702_ALMO/figures/ already exists
2017-03-13 14:08:09,191 - alborex_logger - DEBUG - Figure directory /home/ctroupin/Publis/201702_ALMO/figures/ already exists
2017-03-13 14:08:09,191 - alborex_logger - DEBUG - Figure directory /home/ctroupin/Publis/201702_ALMO/figures/ already exists


## Coastline

In [24]:
reload(alborexdata)

<module 'alborexdata' from '/home/ctroupin/Publis/201703_AlborexData/python/alborexdata.py'>

In [25]:
coastdir = '../data/'
coastfile = os.path.join(coastdir, 'coastline_cartex_f3.txt')

if os.path.exists(coastfile):
    logger.info("{0} exists".format(coastfile))
    loncoast, latcoast = alborexdata.read_lonlat_coast(coastfile)
else:
    logger.warning("{0} does't exist".format(coastfile))  

2017-03-13 14:08:11,981 - alborex_logger - INFO - ../data/coastline_cartex_f3.txt exists
2017-03-13 14:08:11,981 - alborex_logger - INFO - ../data/coastline_cartex_f3.txt exists
2017-03-13 14:08:11,981 - alborex_logger - INFO - ../data/coastline_cartex_f3.txt exists


Create the projection (only once)

In [26]:
m = Basemap(projection='merc', llcrnrlon=coordinates[0], llcrnrlat=coordinates[2],
                urcrnrlon=coordinates[1], urcrnrlat=coordinates[3],
                lat_ts=0.5 * (coordinates[2] + coordinates[3]), resolution='h')

## Loop on the files

In [ ]:
for sstfiles in sstfilelist:
    logger.info("Working on file: {0}".format(os.path.basename(sstfiles)))
    
    # Read data from file
    lon, lat, sst, sstqual, year, day, platform = alborex_functions.load_sst_l2(sstfiles)
    
    # Create date from year and day
    titledate = (datetime.datetime(int(year), 1, 1) + datetime.timedelta(int(day) - 1)).strftime('%Y-%m-%d')
    title = ' '.join(('MODIS', platform, ' ', titledate))
    # logger.debug(title)
    
    # Make the SST when flag > 1
    sst = np.ma.masked_where(sstqual > 1, sst)

    # Create figure name (remove .nc extension and substitute . by _)
    figname = ''.join(('_'.join(os.path.basename(sstfiles).split('.')[:-1]), '.png'))
    logger.info("Making figure {0}".format(figname))
    
    if makeFigs:
        # Normal figures
        fig = plt.figure()
        ax = plt.subplot(111)
        m.ax = ax
        pcm = m.pcolormesh(lon, lat, sst, latlon=True, vmin=17., vmax=20, cmap=cmap)

        # Add the contours
        for i in range(0, len(loncoast)):
            m.plot(np.array(loncoast[i]), np.array(latcoast[i]), color='k', linewidth=.1, latlon=True)

        alborexdata.add_map_grid(m, coordinates, dlon=2., dlat=2.,
                                 fontname='Times New Roman', fontsize=14, 
                                 linewidth=0.2, zorder=1, color=".6")

        #m.fillcontinents(color='w', zorder=2)
        #ax.set_xlim(coordinates[0], coordinates[1])
        #ax.set_ylim(coordinates[2], coordinates[3])

        cb = plt.colorbar(pcm, extend='both', shrink=0.8)
        cb.set_label('$^{\circ}$C', rotation=0, ha='left')
        plt.title(title)
        plt.savefig(os.path.join(figdir, figname))
        plt.close()
        
    if makeFigsLeaflet:
    
        # Figures without border and axes
        m = Basemap(llcrnrlon=coordinates[0],
                    llcrnrlat=coordinates[2],
                    urcrnrlon=coordinates[1],
                    urcrnrlat=coordinates[3], resolution = 'l', epsg=3857)

        llon, llat = m(lon, lat)
        fig = plt.figure(frameon=False)
        ax = fig.add_axes([0, 0, 1, 1])
        m.pcolormesh(llon, llat, sst, vmin=17., vmax=20., cmap=cmap)
        ax.axis('off')
        #ax.set_xlim(lon.min(), lon.max())
        #ax.set_ylim(lat.min(), lat.max())
        f1 = plt.gca()
        f1.axes.get_xaxis().set_ticks([])
        f1.axes.get_yaxis().set_ticks([])
        plt.savefig(os.path.join(figdir2, figname), transparent=True, 
                    bbox_inches='tight', pad_inches=0)
        plt.close()

2017-03-13 14:11:14,518 - alborex_logger - INFO - Working on file: A2015330121500.L2_LAC_SST.nc
2017-03-13 14:11:14,518 - alborex_logger - INFO - Working on file: A2015330121500.L2_LAC_SST.nc
2017-03-13 14:11:14,518 - alborex_logger - INFO - Working on file: A2015330121500.L2_LAC_SST.nc
2017-03-13 14:11:14,995 - alborex_logger - INFO - Making figure A2015330121500_L2_LAC_SST.png
2017-03-13 14:11:14,995 - alborex_logger - INFO - Making figure A2015330121500_L2_LAC_SST.png
2017-03-13 14:11:14,995 - alborex_logger - INFO - Making figure A2015330121500_L2_LAC_SST.png
2017-03-13 14:11:26,247 - alborex_logger - INFO - Working on file: A2015330135000.L2_LAC_SST.nc
2017-03-13 14:11:26,247 - alborex_logger - INFO - Working on file: A2015330135000.L2_LAC_SST.nc
2017-03-13 14:11:26,247 - alborex_logger - INFO - Working on file: A2015330135000.L2_LAC_SST.nc
2017-03-13 14:11:27,473 - alborex_logger - INFO - Making figure A2015330135000_L2_LAC_SST.png
2017-03-13 14:11:27,473 - alborex_logger - INFO 